In [ ]:
import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sb 
%matplotlib inline 

from statsmodels.tsa.ar_model import AutoReg 
from sklearn.metrics import mean_squared_error 
 
# from pandas.plotting import lag_plot, autocorrelation_plot 
# from statsmodels.tsa.seasonal import seasonal_decompose 

In [ ]:
# min_tmp = pd.read_csv('../Datasets/daily-min-temperatures.csv', header=0, parse_dates=[0])  
min_tmp = pd.read_csv('../Datasets/daily-min-temperatures.csv', header=0, parse_dates=[0], index_col=0) 
min_tmp 

In [ ]:
min_tmp.shape 

In [ ]:
if not isinstance(min_tmp.index, pd.DatetimeIndex):
    min_tmp.index = pd.to_datetime(min_tmp.index)
    print('Converted index to DateTime format.') 


In [ ]:
min_tmp.index = min_tmp.index.to_period('D')      # Works with index_col=0 in read_csv 
min_tmp 

Test and Train split - 

In [ ]:
train, test = min_tmp[1: (min_tmp.shape[0]-7)], min_tmp[(min_tmp.shape[0]-7):]  

In [ ]:
train 

In [ ]:
test 

In [ ]:
train_y, test_y = train['Temp'], test['Temp'] 

In [ ]:
print(f" y_train size : {train_y.shape} \n y_test size : {test_y.shape}") 
print(f" y_train : {train_y} \n y_test : {test_y}") 

# Auto-Regression Model -  
(Regression of lagged values) 

In [ ]:
l = int(input("Number of Lags : ")) 
model = AutoReg(train['Temp'], lags=l)    
model_fit = model.fit() 
model_fit 

In [ ]:
# Number of lag variables 

# kar = model_fit.k_ar 
# kar 

In [ ]:
# Coefficient of lag values 
model_fit.params 

In [ ]:
preds = model_fit.predict(start=len(train), end=(len(train) + len(test) - 1))  
preds 

In [ ]:
mse_test = mean_squared_error(test_y, preds) 
print(f"Mean Squared Error (Test) : {mse_test}") 

In [ ]:
plt.plot(test_y.index.to_timestamp(), test_y)  
plt.plot(test_y.index.to_timestamp(), preds, color='red') 

Walk-Forward Validation -  

In [ ]:
train, test = min_tmp.Temp[1: (min_tmp.shape[0]-7)], min_tmp.Temp[(min_tmp.shape[0]-7):]  

In [ ]:
train 

In [ ]:
test 

In [ ]:
# data = train     # Data variable to hold test values for future predictions 
# model = AutoReg(data, lags=l) 

l = int(input("Number of Lags : ")) 
model = AutoReg(train, lags=l) 
model_fit = model.fit() 

prev = list(train)     # Store past observations 
pred_y_val = [] 

for t in test:

    y = model_fit.predict(start=len(prev), end=len(prev)) 
    print(y.values[0]) 
    
    pred_y_val.append(y.values[0]) 
    prev.append(t)  


# for t in test:
#     y = model_fit.predict(start=len(train), end=(len(train) + len(test) - 1)) 
#     print(y.values[0]) 
#     pred_y_val.append(y.values[0]) 

#     data = np.append(data, t) 
#     data = pd.Series(data) 

In [ ]:
pred_y_val 

In [ ]:
test.values 

In [ ]:
mse_test_wf = mean_squared_error(test.values, pred_y_val) 
print(f"Mean Squared Error (Test) : {mse_test_wf}") 

In [ ]:
plt.plot(test.index.to_timestamp(), test.values, label="Actual")
plt.plot(test.index.to_timestamp(), pred_y_val, color='red', linestyle='dashed', label="Predicted")
plt.legend() 
plt.show() 